# COVID-19 Ländervergleich

## Ingo Dahn (dahn@dahn-research.eu)

Mit diesem Notebook kann man die Entwicklung der wöchentlichen COVID-19-Daten seit Anfang 2020 (Infizierte und Verstorbene) in zwei Ländern vergleichen. Diese Seite wird unter der [Creative Commons Lizenz CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) veröffentlicht.

Die Daten werden aktuell von https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/csv geladen.

Für die Zeit bis zum 14.12.2020 stehen auch tagesaktuelle Daten zur Verfügung, deren Auswertung Sie [hier](https://dahn-research.eu/corona/Daily.html) finden.

In [1]:
import pandas as pd
url="https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/csv"
data=pd.read_csv(url)
dataV=data.values
dataT=data.T
dataTV=dataT.values
regions=sorted(set(dataTV[2]))
dataD={}
for n in range(0,len(dataV)):
   line=dataV[n]
   region=line[2];country=line[0];date=line[6]; cases=line[4]; deaths=line[5]; population=line[3]
   if country == 'Cura\xc3\xa7ao':
    country='Curacao'
   if region == 'Other' or population == 'nan':
       next
   if not region in dataD.keys():
               dataD[region]={}
   if not country in dataD[region].keys():
        dataD[region][country]={'population': population, 'cases':[],'deaths':[]}
   if line[4] == 'cases':
       dataD[region][country]['cases'].append((date,line[5]))
   if line[4] == 'deaths':
       dataD[region][country]['deaths'].append((date,line[5]))
defaultCountries={'Europe': 'Germany', 'America': 'United States','Asia': 'China', 'Africa':'South_Africa', 'Oceania': 'Australia'}
datesAll=[ dc[0] for dc in dataD['Europe']['Germany']['cases'] ]
dataNr=len(datesAll)
style = """
<style type="text/css">
.sagecell_interactContainer{border: none; margin: 0px; padding: 0px}
.sagecell_interactControlLabel{min-width: 8ex; text-align: left !important}
.sagecell_interactOutput{padding: 0px}
</style>
"""
show(html(style))
show(html(u"Daten von Woche %s bis Woche %s geladen"%(datesAll[0],datesAll[dataNr-1])))

datesMap={}
for n in range(0,dataNr):
    datesMap[datesAll[n]]=n

regions=sorted(defaultCountries.keys())

def setDates(Data):
    LL=[0 for n in range(0,dataNr)]
    for i in range(0,len(Data)):
        LL[datesMap[Data[i][0]]]=Data[i][1]
    return LL

def cumulateList(L):
    LL=[];s=0;
    for i in range(0,len(L)):
        s += L[i]
        LL.append(s)
    return LL
def normListPer(L,Ord,Pop):
    po=Ord/Pop
    return [n*po for n in L]

def cumulatedNormed(L,c,n,P):
    casePlotData0=L
    if c:
        casePlotData1=cumulateList(L)
    else:
        casePlotData1=casePlotData0
    if n:
        casePlotData=normListPer(casePlotData1,10^5,P)
    else:
        casePlotData=casePlotData1
    return casePlotData

region_1='Europe';country_1='Germany'
region_2='America'; country_2='United States'
@interact
def panel_11(region_1s=selector(regions,default=region_1,buttons=True,label="Region 1:")):
    @interact
    def panel_12(country_1s=selector(sorted(dataD[region_1s].keys()),default=defaultCountries[region_1s], label="Land 1:")):
        global region_1
        region_1=region_1s
        global country_1
        country_1=country_1s

        @interact
        def panel_21(region_2s=selector(regions,default=region_2,buttons=True,label="Region 2:")):
            @interact
            def panel_22(country_2s=selector(sorted(dataD[region_2s].keys()),default=defaultCountries[region_2s], label="Land 2:")):
                global region_2
                region_2=region_2s
                global country_2
                country_2=country_2s
                @interact
                def _(cumulate1=checkbox(False,label='Werte kumulieren'), norm1=checkbox(True,label="Werte normieren")):
                    global cumulate
                    cumulate=cumulate1
                    global norm
                    norm=norm1
                    C1=setDates(dataD[region_1][country_1]['cases'])
                    C2=setDates(dataD[region_2][country_2]['cases'])
                    D1=setDates(dataD[region_1][country_1]['deaths'])
                    D2=setDates(dataD[region_2][country_2]['deaths'])
                    DataCasePlot1=cumulatedNormed(C1,cumulate1,norm1,dataD[region_1][country_1]['population'])
                    DataCasePlot2=cumulatedNormed(C2,cumulate1,norm1,dataD[region_2][country_2]['population'])
                    DataDeathPlot1=cumulatedNormed(D1,cumulate1,norm1,dataD[region_1][country_1]['population'])
                    DataDeathPlot2=cumulatedNormed(D2,cumulate1,norm1,dataD[region_2][country_2]['population'])
                    labelCase1="COVID-19-Infektionen in "+country_1
                    labelCase2="COVID-19-Infektionen in "+country_2
                    labelDeath1="COVID-19-Tote in "+country_1
                    labelDeath2="COVID-19-Tote in "+country_2
                    if cumulate:
                        labelCase1 += " kumuliert";labelCase2 += " kumuliert";
                        labelDeath1 += " kumuliert";labelDeath2 += " kumuliert";
                    if norm:
                        labelCase1 += ", pro 100 000 Einwohner";labelCase2 += ", pro 100 000 Einwohner";
                        labelDeath1 += ", pro 100 000 Einwohner";labelDeath2 += ", pro 100 000 Einwohner";
                    show(list_plot(DataCasePlot1, color='blue',legend_label=labelCase1)+list_plot(DataCasePlot2, color='red',legend_label=labelCase2))
                    show(list_plot(DataDeathPlot1, color='blue',legend_label=labelDeath1)+list_plot(DataDeathPlot2, color='red',legend_label=labelDeath2))

<style type="text/css">
.sagecell_interactContainer{border: none; margin: 0px; padding: 0px}
.sagecell_interactControlLabel{min-width: 8ex; text-align: left !important}
.sagecell_interactOutput{padding: 0px}
</style>

Daten von Woche 2020-01 bis Woche 2021-06 geladen

Interactive function <function panel_11 at 0x7f1e75cbf160> with 1 widget
  region_1s: ToggleButtons(descriptio…